In [192]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scipy.special as special
import time as time
import datetime as dt
import os

from bs4 import BeautifulSoup
import requests 

Yes24 Bestseller Data in Dataframe

In [193]:
# get bestseller links
yes24_url = 'http://www.yes24.com/24/Category/BestSeller' 
result = requests.get(yes24_url)
raw_html = BeautifulSoup(result.text, 'html.parser')

a_html = raw_html.find('ol')
list_yes24_bestseller_data = a_html.find_all('p', class_='copy')

yes24_bestseller_urls = [] 
for i in list_yes24_bestseller_data:
    urls_in_html = i.find('a')
    yes24_bestseller_urls.append('http://www.yes24.com/' + urls_in_html.get('href'))

In [194]:
# set frame for data
data = []

# get data for each best seller, create 2D array
for url in yes24_bestseller_urls: 
    result = requests.get(url) # get html
    raw_html = BeautifulSoup(result.text, 'html.parser')

    title = raw_html.find(name='h2', class_='gd_name').string

    author_wrapper = raw_html.find(name='span', class_='gd_auth')
    author = author_wrapper.find('a').string

    publisher_wrapper = raw_html.find(name='span', class_='gd_pub')
    publisher = publisher_wrapper.find('a').string

    published_date_str = raw_html.find(name='span', class_='gd_date').string
    published_date_num = published_date_str.translate({ord(i): None for i in '월년일'})

    origin_wrapper = raw_html.find(name='span', class_='gd_orgin')
    if origin_wrapper != None:
        origin_wrapper = origin_wrapper.find('a').string

    yes24_review_score = raw_html.find(name='em', class_='yes_b').string
    yes24_review_number = raw_html.find(name='em', class_='txC_blue').string

    yes24_fixed_price = raw_html.find_all(name='em', class_='yes_m')[0]
    yes24_fixed_price = yes24_fixed_price.string
    yes24_selling_price = raw_html.find_all(name='em', class_='yes_m')[1]
    yes24_selling_price = yes24_selling_price.string

    ISBN13_parent_html = raw_html.find(name='th', class_='txt', string='ISBN13').parent
    ISBN13 = ISBN13_parent_html.find(name='td', class_='txt lastCol').string

    ISBN10_parent_html = raw_html.find(name='th', class_='txt', string='ISBN10').parent
    ISBN10 = ISBN10_parent_html.find(name='td', class_='txt lastCol').string

    yes24_sales_text = raw_html.find(name ='span', class_='gd_sellNum')
    yes24_sales_text = yes24_sales_text.text
    yes24_sales_index = ''
    for i in yes24_sales_text:
        if i.isnumeric() == True:
            yes24_sales_index = yes24_sales_index + i

    genre_list = raw_html.find(name='em', class_='bl_dot bgYUI').parent
    yes24_final_genre_name = genre_list.find_all('a')[-1].string

    yes24_bestseller_data = [title, author, publisher, published_date_num, origin_wrapper, yes24_review_score, yes24_review_number, yes24_fixed_price, yes24_selling_price, ISBN13, ISBN10, yes24_sales_index, yes24_final_genre_name]
    data.append(yes24_bestseller_data)

In [195]:
# create dataframe with values
columns=['Title', 'Author', 'Publisher', 'Published_date', 'Origin', 'Yes24_Review_score', 'Yes24_Review_number', 'Yes24_Fixed_price', 'Yes24_Selling_price', 'ISBN13', 'ISBN10', 'Yes24_sales_index', 'Yes24_Final_genre_name'] # change to all caps or all lowercase
number_bestsellers = len(yes24_bestseller_urls)
rank = np.arange(1,number_bestsellers+1)
yes24_dataframe = pd.DataFrame(data=data, index=rank, columns=columns)
yes24_dataframe['Published_date'] = pd.to_datetime(yes24_dataframe['Published_date'], format='%Y %m %d')

Aladin Bestseller Data in CSV

In [196]:
# Get excel file from Aladin, 데이트 타임 활용하기
Aladin_URL = 'https://www.aladin.co.kr/shop/common/wbest_excel.aspx?BestType=Bestseller&BranchType=1&CID=0&Year=2023&Month=2&Week=3' # 링크가 매주 바뀜 

req = requests.get(Aladin_URL) # 200이면 온라인

Aladin_excel_name = '알라딘_주간+베스트_국내도서.csv' # 일반화 & edit later 

with open(Aladin_excel_name, 'wb') as f: # write, binary, with 사용해서 효율적
    for chunk in req.iter_content(chunk_size=9000): # 실제로 파일을 열어서 작성
        if chunk:
            f.write(chunk)

os.rename(r'H:\내 드라이브\연세대학교\3학년 2022 겨울 계절\파이썬_통계_공부하기\Analyze_Bestseller_Project\알라딘_주간+베스트_국내도서.csv', r'H:\내 드라이브\연세대학교\3학년 2022 겨울 계절\파이썬_통계_공부하기\Analyze_Bestseller_Project\aladin_bestseller_data.csv')

Kyobo Bestseller Data in CSV

In [197]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [198]:
# Get excel file from Kyobobook with Selenium
import shutil

Kyobo_URL = "https://product.kyobobook.co.kr/bestseller/online?period=001#?page=1&per=20&ymw=&period=001&saleCmdtClstCode=&dsplDvsnCode=000&dsplTrgtDvsnCode=001&saleCmdtDsplDvsnCode="

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # headless 해결해보기

driver.get(url=Kyobo_URL)
driver.implicitly_wait(time_to_wait=3)

download_excel_button = driver.find_element(By.XPATH, '//*[@id="excel_btn"]')
download_excel_button.click() 

time.sleep(10)

driver.close()

shutil.move(r'C:\Users\seohy\Downloads\교보문고_온라인_베스트셀러_상품리스트.xlsx', r'H:\내 드라이브\연세대학교\3학년 2022 겨울 계절\파이썬_통계_공부하기\Analyze_Bestseller_Project\교보문고_온라인_베스트셀러_상품리스트.xlsx') # 파일 이름에 날짜 붙이기

os.rename(r'H:\내 드라이브\연세대학교\3학년 2022 겨울 계절\파이썬_통계_공부하기\Analyze_Bestseller_Project\교보문고_온라인_베스트셀러_상품리스트.xlsx', r'H:\내 드라이브\연세대학교\3학년 2022 겨울 계절\파이썬_통계_공부하기\Analyze_Bestseller_Project\kyobo_bestseller_data.xlsx')


Bring All Three Files Together 

In [219]:
aladin_path = r"H:\내 드라이브\연세대학교\3학년 2022 겨울 계절\파이썬_통계_공부하기\Analyze_Bestseller_Project\aladin_bestseller_data.csv"
kyobo_path = r"H:\내 드라이브\연세대학교\3학년 2022 겨울 계절\파이썬_통계_공부하기\Analyze_Bestseller_Project\kyobo_bestseller_data.xlsx"

aladin_dataframe = pd.read_csv(aladin_path)
kyobo_dataframe = pd.read_excel(kyobo_path)

c:\Users\seohy\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [200]:
# Clean and reorganize yes24 dataframe
yes24_dataframe['ISBN13'] = yes24_dataframe['ISBN13'].astype(str)
yes24_dataframe = yes24_dataframe.set_index('ISBN13')

yes24_dataframe_columns_new_order = ['Title', 'Author', 'Publisher', 'Published_date', 'Origin', 'Yes24_Review_score', 'Yes24_Review_number', 'Yes24_Fixed_price', 'Yes24_Selling_price', 'Yes24_sales_index', 'Yes24_Final_genre_name']
yes24_dataframe = yes24_dataframe[yes24_dataframe_columns_new_order]


In [220]:
# Clean and organize Aladin Dataframe
aladin_dataframe = aladin_dataframe.drop(index=1000)

aladin_dataframe['ISBN13'] = aladin_dataframe['ISBN13'].astype(str)
aladin_dataframe['ISBN13'] = aladin_dataframe['ISBN13'].str.replace('.0', '', regex=False) # .str으로 value 선택
aladin_dataframe = aladin_dataframe.set_index('ISBN13')

aladin_dataframe = aladin_dataframe.rename(columns = {'정가': 'Aladin_Fixed_price', '판매가': 'Aladin_Selling_price', '마일리지': 'Aladin_Mileage', '세일즈포인트': 'Aladin_Sales_Point'})
picked_aladin_dataframe = aladin_dataframe[['Aladin_Fixed_price', 'Aladin_Selling_price', 'Aladin_Mileage', 'Aladin_Sales_Point']]
picked_aladin_dataframe


,Aladin_Fixed_price,Aladin_Selling_price,Aladin_Mileage,Aladin_Sales_Point
ISBN13,,,,
9791169794930,"20,000","18,000","1,000점",390885.0
9791168473690,"7,200","6,480",360점,531995.0
9791197389450,"17,000","15,300",850점,88070.0
9791198013026,"18,000","16,200",900점,110050.0
9791169305037,"16,000","14,400",800점,110110.0
...,...,...,...,...
9791168809109,"19,000","17,100",950점,2880.0
9791189363208,"16,000","14,400",800점,2730.0
9791192730226,"19,800","17,820",990점,2500.0


In [202]:
# Clean and organize Kyobo Dataframe
kyobo_dataframe = kyobo_dataframe.rename(columns = {'정가': 'Kyobo_Fixed_price', '판매가': 'Kyobo_Selling_price', '적립예정포인트': 'Kyobo_Sales_Point', '분야': 'Kyobo_Genre'})
picked_kyobo_dataframe = kyobo_dataframe[['Kyobo_Fixed_price', 'Kyobo_Selling_price', 'Kyobo_Sales_Point', 'Kyobo_Genre']]
picked_kyobo_dataframe

,Kyobo_Fixed_price,Kyobo_Selling_price,Kyobo_Sales_Point,Kyobo_Genre
0,"17,000","15,300",850,자기계발
1,"18,000","16,200",900,자기계발
2,"19,000","17,100",950,경제/경영
3,"7,200","6,480",360,자기계발
4,"19,000","19,000",570,시그니처 향
...,...,...,...,...
982,"20,000","18,000","1,000",시/에세이
983,"16,000","14,400",800,소설
984,"15,900","14,310",790,인문
985,"18,000","16,200",900,자기계발


In [221]:
aladin_filter = []
for i in aladin_dataframe.index.tolist():
    truth = i in yes24_dataframe.index.tolist()
    aladin_filter.append(truth)
picked_aladin_dataframe = picked_aladin_dataframe[aladin_filter]


# kyobo_filter = [] # Index is not ISBN
# for i in kyobo_dataframe['상품명'].tolist():
#     truth = i in yes24_dataframe['Title'].tolist()
#    kyobo_filter.append(truth)
# picked_kyobo_dataframe = picked_kyobo_dataframe[kyobo_filter]

adding_frames = [yes24_dataframe, picked_aladin_dataframe]
total_bestseller_data = pd.concat(adding_frames, axis=1)

total_bestseller_data.to_excel('total_bestseller_data.xlsx', encoding='CP949')


c:\Users\seohy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
